# Trader Bot with Gate.io API

In this work, crypto data will be imported using Gate.io API and and a trader bot will be created for automated with python.

**See also** Technical Indicators for Financial Analysis with Python (Jupyter): https://github.com/DrFarukAydin/data-science-portfolio/blob/main/financial-tools/technical-indicators-finance.ipynb

**See also** Binance Trader Bot with Python: https://github.com/DrFarukAydin/data-science-portfolio/blob/main/financial-tools/binance-bot.ipynb

**See also** Data Visualization for Financial Analysis with Python (Jupyter): https://github.com/DrFarukAydin/data-science-portfolio/blob/main/financial-tools/data-visualization-finance.ipynb

<br><br>

**WARNING!!**

<li>The strategies used in this section are <b>not investment advices</b>. They are shown for only educational purposes.
<li>The bot makes real transactions. Therefore, be sure to understand them clearly before implementation. Otherwise, you can lose your money.

<br><br>

## 1. Importing Libraries and Data

<br>**Importing Libraries**

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import sqlalchemy
import time
from datetime import datetime
import os
import asyncio
import warnings
warnings.filterwarnings('ignore')
from mplfinance.original_flavor import candlestick_ohlc
import pandas_ta as pta
import logging
from decimal import Decimal as D

**Note** that some libraries should be installed with pip install.

In [4]:
#pip install gate_api

In [7]:
import gate_api
from gate_api.exceptions import ApiException, GateApiException
from gate_api import ApiClient, Configuration, Order, SpotApi

<br>**Creating Connection**

In [9]:
key="ENTER YOUR KEY HERE"
secret="ENTER YOUR SECRET HERE"
host = "https://api.gateio.ws/api/v4"

In [10]:
config = Configuration(key=key, secret=secret, host=host)
spot_api = SpotApi(ApiClient(config))

<br>**Importing Data**

Importing information about currency pair.

In [13]:
spot_api.get_currency_pair("GT_USDT")

{'amount_precision': 3,
 'base': 'GT',
 'buy_start': 0,
 'fee': '0.2',
 'id': 'GT_USDT',
 'min_base_amount': None,
 'min_quote_amount': '1',
 'precision': 4,
 'quote': 'USDT',
 'sell_start': 0,
 'trade_status': 'tradable'}

<br>

Importing the most recent information for currency pair.

In [24]:
spot_api.list_tickers(currency_pair="GT_USDT")

[{'base_volume': '321454.66023446',
  'change_percentage': '2.59',
  'currency_pair': 'GT_USDT',
  'etf_leverage': None,
  'etf_net_value': None,
  'etf_pre_net_value': None,
  'etf_pre_timestamp': None,
  'high_24h': '3.9173',
  'highest_bid': '3.8971',
  'last': '3.8971',
  'low_24h': '3.7',
  'lowest_ask': '3.8972',
  'quote_volume': '1221566.8425295'}]

<br>

Importing open orders for the selected pair.

In [26]:
spot_api.list_orders("MOOV_USDT", status="open")

[{'account': 'spot',
  'amount': '26923.47',
  'auto_borrow': None,
  'auto_repay': None,
  'create_time': '1656515191',
  'create_time_ms': 1656515191296,
  'currency_pair': 'MOOV_USDT',
  'fee': '0',
  'fee_currency': 'USDT',
  'fill_price': '0',
  'filled_total': '0',
  'gt_discount': False,
  'gt_fee': '0',
  'iceberg': '0',
  'id': '174634514262',
  'left': '26923.47',
  'point_fee': '0',
  'price': '0.0215',
  'rebated_fee': '0',
  'rebated_fee_currency': 'MOOV',
  'side': 'sell',
  'status': 'open',
  'text': '101',
  'time_in_force': 'gtc',
  'type': 'limit',
  'update_time': '1656515191',
  'update_time_ms': 1656515191296}]

<br><br><br>

# 2. TRADING BOT

A trading bot can trade automatically when the conditions defined by us satisfied. These conditions can be a fixed price or a dynamic price calculated by technical indicators.

**WARNING!!**

<li>The strategies used in this section are <b>not investment advices</b>. They are shown for only educational purposes.
<li>The bot makes real transactions. Therefore, be sure to understand them clearly before implementation. Otherwise, you can lose your money.

**Bot Structure**

Bot structure may be the most important part of creating a bot. Before coding this should be designed carefully. The basic steps are as follows:

<li> Defining a variable that shows our current situation, i.e., are we ready to buy or sell. This is crucial because we do not want to buy or sell more than once, when our price criterion is met.
<li> Creating a loop for constantly importing the most recent data.
<li> Checking current order situation, i.e., is an open order has been filled.
<li> Analyzing the most recent data to decide our price that we are willing to trade.
<li> If there is no open order, then create an order using "LIMIT" or continue to create an order later when the price criterion is met.
<li> Change our current variable accordingly.
<br><br>

<img src='bot-flowchart.png'/>

**Trading Strategy**

Before making a trading bot, we should decide our strategy, at which price we would buy or sell. There are countless possibilities to decide that. This work focuses on trading bot, thus, we create a simple grid function.

In general, technical indicators are used to determine the target price. Some technical indicators can be found another work in this repository. The link will be provided as soon as it is uploaded.

In [178]:
def price_finder_grid(side, buy_price=3.95, sell_price=3.956):
    
    if side == "buy":
        return buy_price
    if side == "sell":
        return sell_price

The price_finder function is basically a implemantation of grid trade that is buying/selling at given prices.

<br><br> Following this structure, we can make a trader bot. The explanations will be given in the function.

**Important Note:** The bot may not stop unless you force it, for example, by interrupting kernel.

<li> Firstly, I will present a trader bot.
<li> Then, I will make a trading bot that runs forever following the given strategy.

In [175]:
def spot_trade(side = "sell",currency_pair = "GT_USDT", order_amount = "0", price = "0"):

    # Getting currency pairs
    currency = currency_pair.split("_")[1]
    token = currency_pair.split("_")[0]
    
    # Checking validity of parameters.
    order_amount = int(order_amount)

    # Getting information about the selected pair.
    pair = spot_api.get_currency_pair(currency_pair)
    print("\nTrading for currency pair: " + currency_pair)
    
    #Checking minimum amount
    min_amount = pair.min_base_amount
    if min_amount is not None:
        if float(min_amount)>order_amount:
            print("Minimum amount is ",min_amount)
            return


    # make sure balance is enough
    if side == "buy":
        accounts = spot_api.list_spot_accounts(currency=currency)
        available = float(accounts[0].available)
        print("Account available: {} {}".format(available, currency))
        if available < (order_amount*float(price)):
            print("Account balance not enough")
            return
    if side == "sell":
        accounts = spot_api.list_spot_accounts(currency=token)
        available = float(accounts[0].available)
        print("Account available: {} {}".format(available, token))
    
        if available < order_amount:
            print("Account balance not enough")
            return

    
    # Creating order
    order = Order(amount=str(order_amount), price=str(price), side=side, currency_pair=currency_pair)
    print("\nplace a spot {} order in {} with amount {} and price {}".format( order.side, order.currency_pair,
                order.amount, order.price))
    created = spot_api.create_order(order)
    print("order created with id {}, status {}".format( created.id, created.status))
    
    return created


In [168]:
def trader(currency, amount=0, side="buy", commission=0.002):
    
    #define current position
    open_order = False

    
    #checking the parameters are given correctly
    side = side.lower()
    if side not in ["buy", "sell"]:
        print("Please enter a valid 'side' Value")
        return
    if side == "sell":    #This is placed for avoiding error when calculating Profit
        buyprice=0

    commission = float(commission)
    
    print("\nBOT started!\n")
    
    
       
    
    #Loop begins here
    while True:            
        
        tickers = spot_api.list_tickers(currency_pair=currency)
        assert len(tickers) == 1
        last_price = tickers[0].last
        
        # If there is an open order, change current side if filled, cancel it otherwise to be updated later
        if open_order == True:
            
            #Check order status
            order_result = spot_api.get_order(created.id, currency)
            
            #Check if the existing order is filled
            if order_result.status == 'open':
                         
                if order_result.filled_total != '0':
                    print("Order {} filled {} {}, left: {}".format( order_result.id, order_result.filled_total,currency, order_result.left))
                else:
                    cancelled = spot_api.cancel_order(order_id=order_result.id, currency_pair=currency)
                    print("{} order is cancelled.".format(cancelled.currency_pair))
                    open_order = False
                
            else:   #If order is filled
                trades = spot_api.list_my_trades(currency_pair=currency, order_id=created.id)
                for t in trades:
                    print("Order {} filled {} {} with price {}".format( t.order_id, t.amount,currency, t.price))
                    
                if side == "sell":
                    side = "buy"
                elif side == "buy":
                    side = "sell"
                    
                open_order = False
            
    
        
        # Setting the target price we want to buy or sell
        target_price = price_finder_grid(side=side)
        
        
        
        
        # Create a buy/sell order
        if open_order==False:
            created = spot_trade(currency_pair = currency, 
                               side=side,
                              order_amount=str(amount),
                              price=str(target_price))
            
            

            print(datetime.fromtimestamp(time.time()).strftime("%H:%M:%S"),"   //  Last Price:{}\n\n".format(last_price))
            open_order = True

        if created is None:  # Break the function if there is a warning
            print("Bot has stopped!")
            break
           
        #Wait between each loop. 
        #Too many requests from API may result in banning or cutting connection.
        time.sleep(15)

                    
    
        

In [155]:
trader(currency="AZERO_USDT", amount="15", side="buy")


BOT started!

Trading for currency pair: AZERO_USDT
Account available: 5.611614197624 USDT
Account balance not enough
02:19:11    //  Last Price:0.6934


Bot has stopped!


In [180]:
trader(currency="GT_USDT", amount="1", side="sell")


BOT started!


Trading for currency pair: GT_USDT
Account available: 1.996 GT

place a spot sell order in GT_USDT with amount 1 and price 3.956
order created with id 179606664418, status closed
02:39:17    //  Last Price:3.9564


Order 179606664418 filled 1 GT_USDT with price 3.9564

Trading for currency pair: GT_USDT
Account available: 5.603482197624 USDT

place a spot buy order in GT_USDT with amount 1 and price 3.95
order created with id 179606728468, status open
02:39:34    //  Last Price:3.9563


GT_USDT order is cancelled.

Trading for currency pair: GT_USDT
Account available: 5.603482197624 USDT

place a spot buy order in GT_USDT with amount 1 and price 3.95
order created with id 179606798361, status open
02:39:51    //  Last Price:3.9563


GT_USDT order is cancelled.

Trading for currency pair: GT_USDT
Account available: 5.603482197624 USDT

place a spot buy order in GT_USDT with amount 1 and price 3.95
order created with id 179606889879, status open
02:40:08    //  Last Price

KeyboardInterrupt: 

<br><br> The bot is working as expected. I have not implemented an advanced strategy and keep the difference between buy and sell prices too narrow to test all code lines. I made a small loss because the commission was greater than my gross profit. 

<br><br><br>

## 3. CONCLUSION AND DISCUSSION

Trader bots allow users to buy or sell stocks automatically at a given price or a dynamic price. We can implement a strategy to trade with bots. This can be finding a price with trading indicators, giving a selling order after buying order is filled or anything you can think of. My motivation was to eliminate human emotions while trading. 

In this work, I have created a trader bot for binance that makes an actual trade in real life. I explained all steps and created a flowchart to make the process clear. This bot can be improved far more with your imaginations and skills.

**See also** Technical Indicators for Financial Analysis with Python (Jupyter): https://github.com/DrFarukAydin/data-science-portfolio/blob/main/financial-tools/technical-indicators-finance.ipynb

**See also** Binance Trader Bot with Python: https://github.com/DrFarukAydin/data-science-portfolio/blob/main/financial-tools/binance-bot.ipynb

**See also** Data Visualization for Financial Analysis with Python (Jupyter): https://github.com/DrFarukAydin/data-science-portfolio/blob/main/financial-tools/data-visualization-finance.ipynb